In [72]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('seaborn-darkgrid')
import seaborn as sns

# Read in Data

In [34]:
# import county level population density data
df_density = pd.read_csv('Data/county_density.csv').dropna()
df_density['GCT_STUB.display-label'] = df_density['GCT_STUB.display-label'].apply(lambda x: x.replace(' County', ''))
df_density['state_county'] = df_density['GEO.display-label'] + ' ' + df_density['GCT_STUB.display-label']

# import county level mobility data from Google's data reports
df_movement = pd.read_csv('Data/Global_Mobility_Report.csv')
df_movement = df_movement.drop(columns = ['iso_3166_2_code'])
df_movement = df_movement.loc[df_movement['country_region_code'] == 'US']
df_movement = df_movement.dropna().reset_index()
df_movement['sub_region_2'] = df_movement['sub_region_2'].apply(lambda x: x.replace(' County', ''))
df_movement['date_county'] = df_movement['date'] + ' ' + df_movement['sub_region_2'] + ' ' + df_movement['sub_region_1']

# Parse data
df_movement['date'] = pd.to_datetime(df_movement['date'])

# Rename columns
df_movement = df_movement.rename(columns = {
    'retail_and_recreation_percent_change_from_baseline': 'retail',
    'grocery_and_pharmacy_percent_change_from_baseline' : 'grocery',
    'parks_percent_change_from_baseline': 'parks',
    'transit_stations_percent_change_from_baseline' : 'transit',
    'workplaces_percent_change_from_baseline' : 'workplace',
    'residential_percent_change_from_baseline': 'residential'
})

# Add weekdays and process dates
df_movement['weekday'] = df_movement['date'].map(lambda x: x.weekday())
# Get number of days since beginning of time period
start = df_movement['date'][0] # get first data
df_movement['days'] = df_movement['date'].map(lambda x: (x - start).days)

In [78]:
# Save processed data
#TODO - ADD collated case and density data 
df_movement.to_csv('Data/processed_mobility.csv')